![NCAR UCAR Logo](../NCAR_CISL_NSF_banner.jpeg)
# Hands-On Session Using OpenACC in MPAS-A

By: Daniel Howard [dhoward@ucar.edu](mailto:dhoward@ucar.edu), Consulting Services Group, CISL & NCAR 

Date: April 28th 2022

In this notebook, we explore the GPU enabled [MPAS-A](http://mpas-dev.github.io/atmosphere/OpenACC/index.html) (Model Prediction Across Scales-Atmosphere) to apply techniques learned from MiniWeather and implementing OpenACC to develop for GPUs. 

* Review of exercises from prior OpenACC/MiniWeather sessions Part 1 and Part 2
* MPAS-Atmosphere model overview
* Managing GPU data in large software projects
* Assessing performance of extracted GPU kernels in MPAS-A

 Head to the [NCAR JupyterHub portal](https://jupyterhub.hpc.ucar.edu/stable) and __start a JupyterHub session on Casper login__ (or batch nodes using 1 CPU, no GPUs) and open the notebook in `07_HandsOnMPASA/07_HandsOnMPASA.ipynb`. Be sure to clone (if needed) and update/pull the NCAR GPU_workshop directory.

```shell
# Use the JupyterHub GitHub GUI on the left panel or the below shell commands
git clone git@github.com:NCAR/GPU_workshop.git
git pull
```

# Workshop Etiquette
* Please mute yourself and turn off video during the session.
* Questions may be submitted in the chat and will be answered when appropriate. You may also raise your hand, unmute, and ask questions during Q&A at the end of the presentation.
* By participating, you are agreeing to [UCAR’s Code of Conduct](https://www.ucar.edu/who-we-are/ethics-integrity/codes-conduct/participants)
* Recordings & other material will be archived & shared publicly.
* Feel free to follow up with the GPU workshop team via Slack or submit support requests to [support.ucar.edu](https://support.ucar.edu)
    * Office Hours: Asynchronous support via [Slack](https://ncargpuusers.slack.com) or schedule a time with an organizer

## Notebook Setup
Set the `PROJECT` code to a currently active project, ie `UCIS0004` for the GPU workshop, and `QUEUE` to the appropriate routing queue depending on if during a live workshop session (`gpuworkshop`), during weekday 8am to 5:30pm MT (`gpudev`), or all other times (`casper`). Due to limited shared GPU resources, please use `GPU_TYPE=gp100` during the workshop. Otherwise, set `GPU_TYPE=v100` (required for `gpudev`) for independent work. See [Casper queue documentation](https://arc.ucar.edu/knowledge_base/72581396#StartingCasperjobswithPBS-Concurrentresourcelimits) for more info.  

In [ ]:
export PROJECT=UCIS0004
export QUEUE=gpudev
export GPU_TYPE=v100

## Review of MiniWeather Performance Optimization
At the end of last session, it was suggested to use `async` and predominantly `collapse` clauses to achieve optimal performance in MiniWeather kernels. Using `NX=1024` and `NZ=512`, the most expensive kernel in terms of compute time was at [__Line 231__](../05_DirectivesOpenACC/fortran/miniWeather_mpi_openacc.F90#L231) in the `semi_discrete_step` subroutine, with `NVCOMPILER_ACC_TIME` statistics highlighted below:

```shell
/glade/u/home/dhoward/GPU_workshop/05_DirectivesOpenACC/fortran/miniWeather_mpi_exercise2.F90 # Source file with OpenACC kernel code
semi_discrete_step  NVIDIA  devicenum=0              # Name of subroutine from which kernel is launched
time(us): 62,147
257: compute region reached 924 times                # Specific line number for GPU kernel and number times reached/launched
257: kernel launched 924 times
grid: [16384]  block: [128]                          # Arrangement of gang/worker/vector in terms of grids and blocks
device time(us): total=62,147 max=70 min=66 avg=67   # Timing statistics of the GPU kernel
elapsed time(us): total=76,527 max=87 min=80 avg=82  # Timing statistics of the CPU call (less accurate with asynchronous execution)
257: data region reached 1848 times
```

The arrangement of __gang/worker/vector__ units is provided by __grid: [NUM_GANGS]__ and __block: [VECTOR_LENGTH x NUM_WORKERS]__. The number of workers was 1 in the previous case so is omitted.

Running this version with the NVIDIA NSight Systems Profiler (discussed in later session), we can get a visual representation of the model runtime. You can download and view this profile using the [NVIDIA NSight Systems client](https://developer.nvidia.com/nsight-systems) by downloading (`SHIFT` + `RIGHT-CLICK`) [MW_baseline.nsys-rep](MW_baseline.nsys-rep) in this folder.
![Profile of MiniWeather - Baseline](img/Profile_MiniWeather_Baseline.png)

This timeline shows the kernels running on the GPU runtime in the upper __blue__ compute kernels, __pink__ device to host transfers, and __teal__ host to device transfers segments. The lower segments show the CPU runtime in __blue__ compute kernel launches, __red__ data directives/regions, and beige __wait/synchronize__ sections.

The bright blue highlights the most expensive GPU kernel in the `semi_discrete_step` subroutine with the associated launch call from the CPU highlighted earlier in the timeline.

![Profile of MiniWeather - Baseline timeline only](img/Profile_MiniWeather_Baseline_cropped.png)

Since we used `async`, the GPU kernels run right after one another without any kernel launch/exit costs.

If we did not use `async`, the profile would look like this ([MW_noasync.qdrep](MW_noasync.qdrep)) and time would be lost as the CPU waits between every kernel launch/exit.

![Profile of MiniWeather - No async](img/Profile_MiniWeather_noasync.png)

## MiniWeather - Testing different kernel launch configurations and clauses
Recall the final exercise of the [prior MiniWeather session](../05_DirectivesOpenACC/05p2_openACC_miniWeather_Tutorial.ipynb) where we experimented with various launch configurations in the [miniWeather_mpi_exercise2.F90](../05_DirectivesOpenACC/fortran/miniWeather_mpi_exercise2.F90) source file for specific kernels.

__Were you able to achieve any significant speed-up?__
 
The next panels shows statistical results from some launch configuration experiments using parameters `_NX=1024`, `_NZ=512`, and `_SIM_TIME=10` and different clauses in place of `***` for the `semi_discrete_step` subroutine kernel. Note that `NUM_VARS=4`.

```fortran
    !$acc parallel loop *** async
    do ll = 1 , NUM_VARS 
      !$acc loop ***
      do k = 1 , nz
        !$acc loop ***
        do i = 1 , nx
          state_out(i,k,ll) = state_init(i,k,ll) + dt * tend(i,k,ll)
        enddo
      enddo
    enddo
```

1. __Using `worker/vector/seq` on each loop respectively, the profiler shows `grid: [1]  block: [32x4]`. Why is this arrangement the least performant?__

| MiniWeather Kernel L231, `semi_discrete_step` | Total Device Time ($ \mu s$) |
|---|---|
| BaseLine (on V100) - `collapse(3)` auto `vector_length(128)`        | 62,936      |
| clause - `gang/worker/vector` on each loop resepctively     | 852,859      |
| clause - `worker/vector/seq` (Move NUM_VARS innermost, seq)     | 2,271,059      |
| clause - `gang/vector/seq` (Move NUM_VARS innermost, seq)     | 72,584      |

2. __Did you find any better configurations for this or other kernels in MiniWeather? Explain why it performed better.__
3. __Do you trust the compiler to make relatively optimal choices with minimal direction?__

| MiniWeather Kernel L231, `semi_discrete_step` | Total Device Time ($ \mu s$) |
|---|---|
| BaseLine (on V100) - `collapse(3)` auto `vector_length(128)`        | 62,936      |
| clause - `collapse(3) vector_length(32)` | 100,797      | 
| clause - `collapse(3) vector_length(64)` | 63,010      |
| clause - `collapse(3) vector_length(256)` | 62,990      |
| clause - `collapse(3) vector_length(512)` | 63,032      |
| clause - `collapse(3) vector_length(1024)` | 66,458      |

4. __For `tile()`, why do you think the `(32,1,NUM_VARS=4)` clause was closest to the most performant?__
5. __Can you infer the condition that causes the `tile()` clause to produce incorrect results?__ Hint: What is the max warp size?

| MiniWeather Kernel L231, `semi_discrete_step` | Total Device Time ($ \mu s$) |
|---|---|
| BaseLine (on V100) - `collapse(3)` auto `vector_length(128)`        | 62,936      |
| clause - `tile(32,32,NUM_VARS)` __INCORRECT__     | 26,992      |
| clause - `tile(32,32,1)`     | 73,476      |
| clause - `tile(32,8,NUM_VARS)`      | 77,124      |
| clause - `tile(32,1,NUM_VARS)`      | 65,040      |
| clause - `tile(1024,1,1)`      | 67,393      |
| clause - `tile(128,1,NUM_VARS)`      | 66,421      |
| clause - `tile(128,2,NUM_VARS)`      | 74,295      |
| clause - `tile(128,4,NUM_VARS)` __INCORRECT__      | 35,999      |
| clause - `tile(*,*,*)` -> 32,4,32   | 150,374      |

## MPAS-Atmosphere Overview
![Global Voronoi mesh](img/mpas-logo-mesh.jpeg)

We will now look at a real world production model __[MPAS (Model Prediction Across Scales)](https://mpas-dev.github.io/)__, specifically the GPU version of the atmosphere core __MPAS-A__ and how this model leveraged OpenACC to refactor towards GPU devices.

So far, __only the v6.x Atmosphere core has been ported to GPUs__ and is freely available to review via their [website](https://mpas-dev.github.io/atmosphere/OpenACC/index.html) and the stable [v6.x](https://github.com/MPAS-Dev/MPAS-Model/tree/atmosphere/v6.x-openacc) or v7.x [develop-openacc](https://github.com/MPAS-Dev/MPAS-Model/tree/atmosphere/develop-openacc) branches on GitHub. Some work has also been done on the MPAS-Ocean core given this [presentation](https://www.lanl.gov/org/padwp/adx/computational-physics/parallelcomputing/_assets/docs/2020-student-projects/Ashwath_PCSRI_Final_Presentation.pdf) by PhD student Ashwath Venkataraman.

If you'd like a more complete overview of MPAS, how to run the model, and research applications, see the [2021 MPAS Virtual tutorial](https://www.mmm.ucar.edu/mpas-tutorial-agenda) page or the upcoming [2022 joint WRF/MPAS workshop](https://www.mmm.ucar.edu/events/workshops/2022).

* Fully compressible non-hydrostatic equations written in flux form
* Split-Explicit timestepping via 3rd Order Runge-Kutta, see [AMS Paper - Klemp, Skamarock, and Dudhia](https://journals.ametsoc.org/view/journals/mwre/135/8/mwr3440.1.xml)

<img src="img/MPAS-grid_diagram.png" alt="MPAS grid diagram" style="width:300px;"/>

The MPAS-A kernels we will focus on computes coefficients for vertically implicit gravity-wave/acoustic computations needed for each Runge-Kutta timestep. The previosly linked paper, specifically section 2 and the appendix, covers this in depth with a broader overview given in the 2021 tutorial [Time Integration](https://www2.mmm.ucar.edu/projects/mpas/tutorial/Virtual2021/MPAS_dynamics_time_integration.pdf) presentation.

__However, understanding the numerical physics at play is not required to port well written code to GPUs.__

## Development Process of MPAS-A
![Development Process](img/DevelopmentProcess.png)
Courtesy of Raghu Raj Kumar, NVIDIA

Identifying an established iterative process for GPU development ahead of work performed significantly eases development cost and increases success outcomes.

1. __Establish a baseline__, ensure working and accurate configuration with target hardware and external software.
2. __Port the code__, using incremental addition of OpenACC, perhaps using tools for kernel extraction like [KGen](https://github.com/NCAR/KGen) (Fortran only) to allow separation of concerns.
    * See [KGen Guide](https://ncar.github.io/kgendocs/overview.html) if interested
3. __Optimize computationally expensive kernels__ individually via an analysis and profiling iterative process.
4. __Check portability expectations__ are met and that code satisfies both CPU and GPU unit tests. 
    * Look for and eliminate any __GPU anti-patterns__ such as linked lists data structures or global memory variables which may cause excessive data movement.
    * Repeat Steps 2-4 as needed.
5. __Integrate changes into benchmarks and verification suite__, utilizing version control and ideally a continuous integration process.

#### MPAS-A Performance Baseline
Getting an accurate baseline helps inform where to dedicate development effort. This can be measured using internal timing metrics or your preferred CPU profiler (like [TAU](https://www.cs.uoregon.edu/research/tau/home.php), [Arm Forge Map](https://developer.arm.com/tools-and-software/server-and-hpc/debug-and-profile/arm-forge/arm-map), _gprof_, etc), to __identifiy hotspots__ in the code.
![MPAS Execution Time Baseline](img/MPAS_Baseline.png)

From KISTI, Kim, Kang, & Joh [GPU Acceleration of MPAS Physics
Schemes Using OpenACC](https://www2.cisl.ucar.edu/sites/default/files/2021-10/KISTI%20-%20Joh%2C%20Kang%2C%20%26%20Kim.pdf)

Specific dynamics/physics schemes were prioritized for GPU while some set for CPU.

![CPU GPU Scheme Distribution](img/CPU_GPU_Schemes.png)

A [lagged computation of radiation](https://mpas-dev.github.io/atmosphere/OpenACC/lagged_radiation.html) was established in order to utilize idle CPUs. Requires __manual tuning of load balancing__ between number of CPU and GPU tasks.

![Lagged Radiation Timeline](img/lagged_radiation.svg)

## Managing GPU Data in MPAS-A
Recall that using `!$acc kernels ...` and similar directives will generate lists of variables needed to manage data movement for each compute region.
```shell
153, Generating implicit copyin(rdzu(:)) [if not already present]
         Generating implicit copyout(cofwr(:,:)) [if not already present]
         Generating implicit copyin(p(:,:)) [if not already present]
         Generating implicit copyout(cofwz(:,:)) [if not already present]
         Generating implicit copyin(fzp(:),t(:,:),zz(:,:),fzm(:),cqw(:,:)) [if not already present]
         Generating implicit copyout(coftz(:,:)) [if not already present]
```
These lists can be used and leveraged for your own data directives as GPU development progresses. 

![Iterative Development from kernels to optimized OpenACC](img/GPU_Data_Iterations.png)

Given ported kernels, MPAS-A was designed to create CPU and GPU data copies [at initialization](https://github.com/MPAS-Dev/MPAS-Model/blob/atmosphere/v6.x-openacc/src/core_atmosphere/physics/mpas_atmphys_vars.F) via `!$acc declare create(...)` and copy data at [unstructured data regions](https://github.com/MPAS-Dev/MPAS-Model/blob/atmosphere/v6.x-openacc/src/framework/mpas_pool_routines.F) via `!$acc enter data copyin(...)` prior to each kernel call. Then, each kernel would only require a `present(...)` clause using the prior variable lists. __Reference counters__ would mitigate excessive copies.

```fortran
!!! From mpas_atmphys_vars.F module
real(kind=RKIND),dimension(:,:,:),allocatable:: &
!... arrays related to u- and v-velocities interpolated to theta points:
    u_p,              &!u-velocity interpolated to theta points            [m/s]
    v_p                !v-velocity interpolated to theta points            [m/s]
!$acc declare create(u_p, v_p)
```

Any lingering excessive data copies could be identified by profilers and fixed while other required copies for CPU algorithms & I/O were managed by `!$acc update` directives.

## MPAS-A Kernel Extraction
We will focus on the `atm_compute_vert_imp_coefs_work` subroutine and kernels as extracted by [Supreeth Suresh](https://staff.ucar.edu/users/ssuresh), TDD/ASAP in CISL. This is the [link, Line 2641](https://github.com/MPAS-Dev/MPAS-Model/blob/ff0e97f8de30c06adbc751f3808f246b33281dd0/src/core_atmosphere/dynamics/mpas_atm_time_integration.F#L2641) to the source subroutine in the full model codebase and in this workshop directory is the the extracted set of kernels [mpas_atm_compute_vert_imp_coefs_work.F90](mpas_atm_compute_vert_imp_coefs_work.F90).

Assuming data locality is resolved, this extracted kernel simply utilizes __randomized input data__ as we will be __focusing on optimizing the performance__ of the subroutine's kernels. The kernel is run in a repeating loop so we can get a relatively consistent average of measured performance. A validation tool has not been included at this time but is typically highly recommended.

For large codebases, building and/or using an automated tool like NCAR's [KGen](https://github.com/NCAR/KGen) for Fortran codes or [Kernel Tuner](https://github.com/benvanwerkhoven/kernel_tuner) from NL eScience Center for CUDA/OpenCL codes will likely speed up the development/optimization process.

## EXERCISE: MPAS-A Kernel Optimization
Open the [mpas_atm_compute_vert_imp_coefs_work.F90](mpas_atm_compute_vert_imp_coefs_work.F90) source file and convert the `!$acc kernels` loops to optimized `!$acc parallel ...` compute constructs. Analyze each set of loops and apply appropriate sets of kernel configuraton clauses to achieve improved performance. Note: `!DIR$ IVDEP` tells compiler to ignore loop dependencies for serial vector SIMD compilations.

You are encouraged to reference the initial attempts at optimization done by the `!$acc kernels` directive output during the compilation process. Data management has already been done for you using `-gpu=managed` and `present(var-list)`/`create(var-list)` clauses.

__Record results of your optimization experiments on a chosen kernel__ and try to determine optimal configurations for that kernel. Compare your achieved performance with the [original at Line 2641](https://github.com/MPAS-Dev/MPAS-Model/blob/ff0e97f8de30c06adbc751f3808f246b33281dd0/src/core_atmosphere/dynamics/mpas_atm_time_integration.F#L2641). Work on other kernels as time allows. Note that most kernels may benefit from similar clause specifications since they operate on similar domain sizes/variables.

In [ ]:
module load nvhpc/22.2 &> /dev/null
export _OPENACC=true
make

In [ ]:
qcmd -A $PROJECT -q $QUEUE -l select=1:ncpus=1:ngpus=1 -l gpu_type=$GPU_TYPE -l walltime=20 -v NVCOMPILER_ACC_TIME=1 -- \
`pwd`/vert_implicit_coefs.exe

| MPAS-A Kernels L### | Device Time ($ \mu s $) |
|---|---|
| BaseLine (on V100) - `!$acc kernels` | XX      |
| clause - `gang/vector`               | XX      |
| clause - `tile(##,##)`               | XX      |
| clause - `tile(*,*)`                 | XX      |
| clause - vector_length(XX)           | XX      |
| clause - num_workers(XX)             | XX      |
| ...                                  | XX      |

## Final Points
1. Plan for and __commit to a defined iterative GPU development process__ to remove pain points and manage long term goals of your code project
    * __Smaller, validated incremental changes__ are easier to debug
2. __Start with descriptive__ `!$acc kernels` __then add prescriptive__ `!$acc parallel ...` kernels as needed for expensive kernels
    * `!$acc kernels` can still achieve meaningful performance alone
3. Understand that the GPU development process takes time and effort but __specific tools/techniques can drastically speed up development time__.

## Suggested Resources
* [2021 MPAS Virtual tutorial](https://www.mmm.ucar.edu/mpas-tutorial-agenda)
* Computers & Geosciences, [GPU acceleration of MPAS microphysics WSM6 using OpenACC directives: Performance and verification](https://www.sciencedirect.com/science/article/pii/S0098300420306051) by J. Kim, J. Kang, and M. Joh (KISTI)
* OpenACC.org and NVIDIA managed GitHub, presentations, and learning materials [GPU Bootcamps](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc)
    * Lab sequence on [OpenACC](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc/openacc)
    * Lab sequence on [Profiling Tools with MiniWeather](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc/miniprofiler) 
    * Lab sequence on [Various GPU Programming Paradigms (CUDA, OpenACC, stdPar, OpenMP)](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc/nways)
    * Lab sequence on [Multi-GPU Programming](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc/multi_gpu_nways)
    * Lab sequences on [GPU AI with CFD, PINNs, and Climate models](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc_ai)
    
After this session, we will have three weeks until the next workshop. Order of upcoming sessions will also be adjusted to accomodate availability of a NVIDIA engineer to present on Multi-GPU programming. Look out for upcoming announcements.